# Proyecto del grupo 3

## PARTICIPANTES

* Carla Fernandez
* Jesús Ramírez
* Moisés Gómez 

In [3]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("./project_dataset/extract - cash request - data analyst.csv")